# SARIMAX Forecasting: Train on `train.csv`, Forecast on `test.csv`

**Workflow**:
1. Import libraries.  
2. Configure paths & parameters.  
3. Read `train.csv` (with `sales`) and `test.csv` (future dates, no `sales`).  
4. Loop over each product family:  
   - Aggregate daily `sales` & `onpromotion` from `train.csv`.  
   - (Optional) Hold out last `N_VALID` days of `train.csv` for sanity‐check metrics.  
   - Fit SARIMAX on full `train.csv` series (log1p‐transformed), selected via `auto_arima`.  
   - Save model to `models2/`.  
   - Forecast exactly on the dates present in `test.csv` (using its `onpromotion`).  
5. (Optional) Compute RMSE/MAE/R² on held‐out portion of `train.csv` and save to `performance_results2.csv`.  
6. Save all forecasts (for `test.csv`) into `forecasts2.csv`.

In [1]:
import os
import warnings

import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX, SARIMAXResults
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pmdarima import auto_arima

# Suppress warnings (optional)
warnings.filterwarnings("ignore")

## 1. CONFIGURABLE PARAMETERS

In [ ]:
# (A) Input file paths (adjust if needed)
TRAIN_CSV = "../Data/DM/train.csv"   # Must contain columns: [id, date, store_nbr, family, sales, onpromotion]
TEST_CSV  = "../Data/DM/test.csv"    # Must contain columns: [id, date, store_nbr, family, onpromotion]

# (B) Output directories / filenames
ALL_MODELS_DIR = os.path.join("Models","models2")
os.makedirs(ALL_MODELS_DIR, exist_ok=True)

PERF_DIR    = "Performance"
FORE_DIR    = "Forecasts"
os.makedirs(PERF_DIR, exist_ok=True)
os.makedirs(FORE_DIR, exist_ok=True)

PERF_CSV     = "performance_results2.csv"  # hold-out metrics file (if N_VALID > 0)
FORECAST_CSV = "forecasts2.csv"            # final forecast on test.csv dates

# (C) Hold-out for sanity-check on train.csv
#     If you want to skip hold-out, set N_VALID = 0
N_VALID = 14

# (D) Time-series frequency
#     'D' = daily, 'W' = weekly, 'M' = monthly, etc.
FREQ = "D"

# (E) auto_arima arguments (alter if your data has different seasonality)
AUTO_ARIMA_ARGS = {
    "seasonal": True,       # assume weekly seasonality
    "m": 7,                 # 7 days → weekly cycle
    "start_p": 0, "start_q": 0, "max_p": 5, "max_q": 5,
    "start_P": 0, "start_Q": 0, "max_P": 2, "max_Q": 2,
    "d": None, "D": None,
    "trace": False,
    "error_action": "ignore",
    "suppress_warnings": True,
    "stepwise": True,
    "information_criterion": "aic"
}

## 2. READ TRAIN.CSV & TEST.CSV

In [3]:
print("Loading train.csv and test.csv...")

# A) Load train.csv (with 'sales' and 'onpromotion')
df_train_all = pd.read_csv(TRAIN_CSV, parse_dates=["date"])
#   Expect columns: [id, date, store_nbr, family, sales, onpromotion]

# B) Load test.csv (only onpromotion; no 'sales')
df_test_all = pd.read_csv(TEST_CSV, parse_dates=["date"])
#   Expect columns: [id, date, store_nbr, family, onpromotion]

print(f"→ train.csv: {df_train_all.shape[0]} rows")
print(f"→ test.csv : {df_test_all.shape[0]} rows\n")

# Extract unique product families from train.csv
unique_families = df_train_all["family"].unique()
print(f"Found {len(unique_families)} unique families.\n")

Loading train.csv and test.csv...
→ train.csv: 3000888 rows
→ test.csv : 28512 rows

Found 33 unique families.



## 3. LOOP THROUGH EACH FAMILY → TRAIN on train.csv, SAVE MODEL, FORECAST test.csv

In [4]:
performance_records = []
forecast_records    = []

for family in unique_families:
    print("=======================================================")
    print(f"Processing family = {family}")

    # 1) Filter train.csv for this family (ALL stores together), then aggregate
    df_f = df_train_all[df_train_all["family"] == family].copy()
    if df_f.empty:
        print(f"  No rows for family '{family}' in train.csv. Skipping.\n")
        continue

    # Aggregate sales & onpromotion by date (summing across all stores)
    df_f_agg = (
        df_f
        .groupby("date")[["sales", "onpromotion"]]
        .sum()
        .rename(columns={"onpromotion": "onpromo"})
        .sort_index()
    )

    # Convert to daily PeriodIndex and fill missing dates with 0
    df_f_agg.index = pd.DatetimeIndex(df_f_agg.index).to_period(FREQ)
    df_f_agg = df_f_agg.asfreq(FREQ)
    df_f_agg["sales"]   = df_f_agg["sales"].fillna(0).astype(float)
    df_f_agg["onpromo"] = df_f_agg["onpromo"].fillna(0).astype(float)

    # If not enough points to hold out, skip
    if len(df_f_agg) < (N_VALID + 1):
        print(f"  Only {len(df_f_agg)} points for {family} (< N_VALID+1). Skipping.\n")
        continue

    # 2) (Optional) Hold out last N_VALID days for sanity-check
    if N_VALID > 0:
        df_valid_holdout = df_f_agg.iloc[-N_VALID:]
        df_train_series  = df_f_agg.iloc[: -N_VALID]
        print(f"  Held out {N_VALID} days from train for validation.")
    else:
        df_valid_holdout = None
        df_train_series  = df_f_agg

    # 3) Log-transform the aggregated sales
    df_train_series["y_log"] = np.log1p(df_train_series["sales"])

    # 4) Prepare exogenous for the model (only “onpromo”, no store_nbr)
    exog_train = df_train_series["onpromo"]
    exog_valid = df_valid_holdout["onpromo"] if df_valid_holdout is not None else None

    # 5) Auto-arima on log1p‐sales
    print("  Running auto_arima for hyperparameter tuning...")
    try:
        mi = auto_arima(
            df_train_series["y_log"],
            exogenous = exog_train.values.reshape(-1, 1),
            **AUTO_ARIMA_ARGS
        )
        order_opt          = mi.order
        seasonal_order_opt = mi.seasonal_order
        print(f"   → Selected order = {order_opt}, seasonal_order = {seasonal_order_opt}")
    except Exception as e:
        print(f"   ❗ auto_arima failed for {family}: {e}\n")
        continue

    # 6) Fit SARIMAX on the entire (or held-out) log1p‐series, ignoring store_nbr
    print("  Fitting SARIMAX on the training series (aggregated across stores)...")
    try:
        model = SARIMAX(
            df_train_series["y_log"],
            exog = exog_train,
            order = order_opt,
            seasonal_order = seasonal_order_opt,
            enforce_stationarity = False,
            enforce_invertibility = False
        )
        sarimax_fit = model.fit(disp=False)
    except Exception as e:
        print(f"   ❗ SARIMAX fit error for {family}: {e}\n")
        continue

    # 7) Save the model (.pkl) under all_models/—one model per family
    safe_name = family.replace(" ", "_").replace("/", "_").replace("\\", "_")
    model_file = f"sarimax_family_{safe_name}.pkl"
    model_path = os.path.join(ALL_MODELS_DIR, model_file)
    try:
        sarimax_fit.save(model_path)
        print(f"   → Saved model to {model_path}")
    except Exception as e:
        print(f"   ❗ Could not save model for {family}: {e}")

    # 8) (Optional) Evaluate hold-out performance if N_VALID > 0
    if df_valid_holdout is not None:
        print("  Computing hold-out metrics on train hold-out...")
        try:
            pred_log_valid = sarimax_fit.get_forecast(
                steps=N_VALID,
                exog = exog_valid
            ).predicted_mean
            pred_valid = np.expm1(pred_log_valid)
            actual_valid = df_valid_holdout["sales"].values

            rmse_val = np.sqrt(mean_squared_error(actual_valid, pred_valid))
            mae_val  = mean_absolute_error(actual_valid, pred_valid)
            r2_val   = r2_score(actual_valid, pred_valid)
            print(f"   → Hold-out metrics: RMSE={rmse_val:.2f}, MAE={mae_val:.2f}, R²={r2_val:.3f}")
            performance_records.append({
                "family": family,
                "RMSE": rmse_val,
                "MAE": mae_val,
                "R2": r2_val
            })
        except Exception as e:
            print(f"   ❗ Hold-out forecasting error for {family}: {e}")

    # 9) Forecast on test.csv dates—but first aggregate “onpromotion” by date across all stores
    df_test_sub = df_test_all[df_test_all["family"] == family].copy()
    if df_test_sub.empty:
        print(f"  No rows for {family} in test.csv. Skipping forecast.\n")
        continue

    # Aggregate exogenous (“onpromotion”) by date so we have exactly one row per date
    df_test_sub["date"] = pd.to_datetime(df_test_sub["date"])
    agg_exog = (
        df_test_sub
        .groupby("date")[["onpromotion"]]
        .sum()
        .rename(columns={"onpromotion": "onpromo"})
    )

    # Convert agg_exog.index to PeriodIndex and reindex to daily freq
    agg_exog.index = pd.DatetimeIndex(agg_exog.index).to_period(FREQ)
    agg_exog = agg_exog.asfreq(FREQ, fill_value=0)

    # Forecast “h” points, where h = number of unique dates in agg_exog
    h = len(agg_exog)
    print(f"  Forecasting {h} future points on test.csv for {family}...")
    try:
        pred_log_test = sarimax_fit.get_forecast(
            steps = h,
            exog = agg_exog["onpromo"]
        ).predicted_mean
        pred_sales_test = np.expm1(pred_log_test)
    except Exception as e:
        print(f"   ❗ Forecast error on test for {family}: {e}\n")
        continue

    # 10) Create a date‐level forecast DataFrame and then “broadcast” it back to each store row
    df_forecast_dates = pd.DataFrame({
        "date": agg_exog.index.to_timestamp(),    # e.g. Timestamp("2025-06-01")
        "predicted_sales": pred_sales_test.values
    })

    # Merge these date‐level predictions back onto every row of df_test_sub
    df_test_sub = df_test_sub.merge(
        df_forecast_dates,
        left_on = "date",
        right_on = "date",
        how = "left"
    )

    # Finally, keep store_nbr and id so that each branch gets the same per‐family forecast
    df_forecast_f = pd.DataFrame({
        "id": df_test_sub["id"].values,
        "date": df_test_sub["date"].values,
        "store_nbr": df_test_sub["store_nbr"].values,
        "family": family,
        "predicted_sales": df_test_sub["predicted_sales"].values
    })

    forecast_records.append(df_forecast_f)
    print(f"  → Done forecasting for {family}.\n")

print("=======================================================")
print("All families processed.\n")

Processing family = AUTOMOTIVE
  Held out 14 days from train for validation.
  Running auto_arima for hyperparameter tuning...
   → Selected order = (0, 1, 2), seasonal_order = (1, 0, 1, 7)
  Fitting SARIMAX on the training series (aggregated across stores)...
   → Saved model to models2\sarimax_family_AUTOMOTIVE.pkl
  Computing hold-out metrics on train hold-out...
   → Hold-out metrics: RMSE=58.11, MAE=44.71, R²=0.498
  Forecasting 16 future points on test.csv for AUTOMOTIVE...
  → Done forecasting for AUTOMOTIVE.

Processing family = BABY CARE
  Held out 14 days from train for validation.
  Running auto_arima for hyperparameter tuning...
   → Selected order = (2, 1, 1), seasonal_order = (1, 0, 1, 7)
  Fitting SARIMAX on the training series (aggregated across stores)...
   → Saved model to models2\sarimax_family_BABY_CARE.pkl
  Computing hold-out metrics on train hold-out...
   → Hold-out metrics: RMSE=4.88, MAE=4.32, R²=-0.536
  Forecasting 16 future points on test.csv for BABY CARE

## 4a. SAVE HOLD-OUT PERFORMANCE METRICS TO CSV

In [5]:
if N_VALID > 0:
    perf_df = pd.DataFrame(performance_records)
    perf_df.to_csv(PERF_CSV, index=False)
    print(f"Hold-out performance metrics saved to {PERF_CSV}")
    display(perf_df)
else:
    print("N_VALID = 0 → Skipping hold-out metric computation.")

Hold-out performance metrics saved to performance_results2.csv


,family,RMSE,MAE,R2
0,AUTOMOTIVE,58.105265,44.713942,0.498031
1,BABY CARE,4.881954,4.315050,-0.536127
2,BEAUTY,57.734682,43.596750,0.454078
3,BEVERAGES,28811.946487,19974.961178,-0.134400
4,BOOKS,0.737549,0.612317,-0.025192
5,BREAD/BAKERY,6943.318651,6243.887310,-2.679452
6,CELEBRATION,90.108367,79.652861,0.587997
7,CLEANING,30990.956779,28762.176086,-6.162457
8,DAIRY,9052.795142,8257.368720,-0.995446
9,DELI,1780.059504,1504.723412,0.504561


## 4b. SAVE ALL FORECASTS (test.csv predictions) TO CSV

In [6]:
if forecast_records:
    all_forecasts_df = pd.concat(forecast_records, axis=0, ignore_index=True)
    # Optional: sort by 'id' or by ['family','date']
    all_forecasts_df = all_forecasts_df.sort_values("id").reset_index(drop=True)

    all_forecasts_df.to_csv(FORECAST_CSV, index=False)
    print(f"All forecasts saved to {FORECAST_CSV}")
    display(all_forecasts_df.head(20))
else:
    print("No forecasts generated (check if test.csv had matching families).")

All forecasts saved to forecasts2.csv


,id,date,store_nbr,family,predicted_sales
0,3000888,2017-08-16,1,AUTOMOTIVE,319.731174
1,3000889,2017-08-16,1,BABY CARE,12.459296
2,3000890,2017-08-16,1,BEAUTY,585.071055
3,3000891,2017-08-16,1,BEVERAGES,217954.216139
4,3000892,2017-08-16,1,BOOKS,0.460009
5,3000893,2017-08-16,1,BREAD/BAKERY,35011.358832
6,3000894,2017-08-16,1,CELEBRATION,673.187521
7,3000895,2017-08-16,1,CLEANING,107843.269843
8,3000896,2017-08-16,1,DAIRY,57912.902679
9,3000897,2017-08-16,1,DELI,16790.988552
